In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import tf_keras
from keras import layers
from transformers import RobertaTokenizerFast, RobertaConfig, TFRobertaModel

gpu = 1
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[gpu], 'GPU')
tf.config.experimental.set_memory_growth(gpus[gpu], True)

2024-12-04 17:53:53.455484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733360033.469329 1622800 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733360033.473533 1622800 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 17:53:53.489043: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Create Model

In [ ]:
model_name = 'microsoft/codebert-base'
config = RobertaConfig.from_pretrained(model_name)
config.output_hidden_states = False
tokenizer = RobertaTokenizerFast.from_pretrained(model_name, config=config)
transformer_model = TFRobertaModel.from_pretrained(model_name, config=config)

I0000 00:00:1733360038.553057 1622800 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78101 MB memory:  -> device: 1, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:c1:00.0, compute capability: 8.0
All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [ ]:
bert = transformer_model.layers[0]

# Create input layer for tokenized data
input_ids = tf_keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
attention_mask = tf_keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32') 
inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}
# inputs = {'input_ids': input_ids}

# Load the bert model as a layer
bert_model = bert(inputs)[1]
dropout = tf_keras.layers.Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)

outputs = tf_keras.layers.Dense(units=5, activation="softmax", kernel_initializer=tf_keras.initializers.TruncatedNormal(stddev=config.initializer_range), name='python')(pooled_output)
model = tf_keras.models.Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')

model.summary()

Model: "BERT_MultiLabel_MultiClass"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 attention_mask (InputLayer  [(None, 512)]                0         []                            
 )                                                                                                
                                                                                                  
 input_ids (InputLayer)      [(None, 512)]                0         []                            
                                                                                                  
 roberta (TFRobertaMainLaye  TFBaseModelOutputWithPooli   1246456   ['attention_mask[0][0]',      
 r)                          ngAndCrossAttentions(last_   32         'input_ids[0][0]']           
                             hidden_state=(None, 512, 7                  

# Load and process data

In [ ]:
splits = {'java_train': 'data/java_train-00000-of-00001.parquet', 'java_test': 'data/java_test-00000-of-00001.parquet', 'python_train': 'data/python_train-00000-of-00001.parquet', 'python_test': 'data/python_test-00000-of-00001.parquet', 'pharo_train': 'data/pharo_train-00000-of-00001.parquet', 'pharo_test': 'data/pharo_test-00000-of-00001.parquet'}

python_train = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_train"])
python_test = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_test"])

In [ ]:
def tokenize_data(df, tokenizer):
    # Tokenize data and truncate/pad to 512
    inputs = tokenizer(df, padding='max_length', truncation=True, max_length=512)
    return inputs

# Tokenize all data
python_train_inputs = tokenize_data(python_train.combo.tolist(), tokenizer)
python_test_inputs = tokenize_data(python_test.combo.tolist(), tokenizer)

In [ ]:
# store labels in list form
python_train_labels = python_train.labels.tolist()
python_test_labels = python_test.labels.tolist()

In [ ]:
# Create dataset from the preprocessed data
def create_tf_dataset(inputs, labels):
    return tf.data.Dataset.from_tensor_slices((dict(inputs), labels))

python_train_dataset = create_tf_dataset(python_train_inputs, python_train_labels)
python_test_dataset = create_tf_dataset(python_test_inputs, python_test_labels)

# Compile and train

In [ ]:
# Compile the model with RoBERTa base frozen
model.layers[2].trainable = False

optimizer = tf_keras.optimizers.AdamW(learning_rate=5e-5, epsilon=1e-8)
loss = tf_keras.losses.CategoricalCrossentropy(from_logits = True)
metric = [tf_keras.metrics.CategoricalAccuracy('accuracy'), tf_keras.metrics.TruePositives(), tf_keras.metrics.TrueNegatives(), tf_keras.metrics.FalsePositives(), tf_keras.metrics.FalseNegatives()]

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
model.summary()

batch_size = 8

Model: "BERT_MultiLabel_MultiClass"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 attention_mask (InputLayer  [(None, 512)]                0         []                            
 )                                                                                                
                                                                                                  
 input_ids (InputLayer)      [(None, 512)]                0         []                            
                                                                                                  
 roberta (TFRobertaMainLaye  TFBaseModelOutputWithPooli   1246456   ['attention_mask[0][0]',      
 r)                          ngAndCrossAttentions(last_   32         'input_ids[0][0]']           
                             hidden_state=(None, 512, 7                  

In [ ]:
# Shuffle and batch the training data use autotune for faster execution
train_dataset = python_train_dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

model.fit(train_dataset, epochs=5)

Epoch 1/5


/top/students/UNGRAD/Other/jecircen/home/.conda/envs/tf440/lib/python3.10/site-packages/tf_keras/src/backend.py:5577: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
I0000 00:00:1733360066.488272 1684155 service.cc:148] XLA service 0x7fb098c95a50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733360066.488295 1684155 service.cc:156]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2024-12-04 17:54:26.493346: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1733360066.509969 1684155 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1733360066.578667 1684155 device_compiler.h:188] Compiled cluster using XLA!  Th

236/236 [==============================] - 33s 105ms/step - loss: 1.7025 - accuracy: 0.3047 - true_positives: 0.0000e+00 - true_negatives: 7370.0000 - false_positives: 0.0000e+00 - false_negatives: 2050.0000
Epoch 2/5
236/236 [==============================] - 25s 105ms/step - loss: 1.6869 - accuracy: 0.2999 - true_positives: 0.0000e+00 - true_negatives: 7370.0000 - false_positives: 0.0000e+00 - false_negatives: 2050.0000
Epoch 3/5
236/236 [==============================] - 25s 105ms/step - loss: 1.6842 - accuracy: 0.3259 - true_positives: 0.0000e+00 - true_negatives: 7370.0000 - false_positives: 0.0000e+00 - false_negatives: 2050.0000
Epoch 4/5
236/236 [==============================] - 16s 69ms/step - loss: 1.6841 - accuracy: 0.3174 - true_positives: 0.0000e+00 - true_negatives: 7370.0000 - false_positives: 0.0000e+00 - false_negatives: 2050.0000
Epoch 5/5
236/236 [==============================] - 11s 48ms/step - loss: 1.6779 - accuracy: 0.3163 - true_positives: 0.0000e+00 - true_ne

In [ ]:
#Set RoBERTa base as trainable and retrain (lower learning rate and more epochs)
model.layers[2].trainable = True

optimizer = tf_keras.optimizers.AdamW(learning_rate=5e-5, epsilon=1e-8)
loss = tf_keras.losses.CategoricalCrossentropy(from_logits = True)
metric = [tf_keras.metrics.CategoricalAccuracy('accuracy'), tf_keras.metrics.TruePositives(), tf_keras.metrics.TrueNegatives(), tf_keras.metrics.FalsePositives(), tf_keras.metrics.FalseNegatives()]

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model.fit(train_dataset, epochs=10)

Epoch 1/10
236/236 [==============================] - 86s 252ms/step - loss: 1.3456 - accuracy: 0.5308 - true_positives_1: 637.0000 - true_negatives_1: 7219.0000 - false_positives_1: 151.0000 - false_negatives_1: 1413.0000
Epoch 2/10
236/236 [==============================] - 58s 247ms/step - loss: 0.9475 - accuracy: 0.7091 - true_positives_1: 1170.0000 - true_negatives_1: 7165.0000 - false_positives_1: 205.0000 - false_negatives_1: 880.0000
Epoch 3/10
236/236 [==============================] - 58s 246ms/step - loss: 0.7026 - accuracy: 0.8041 - true_positives_1: 1427.0000 - true_negatives_1: 7221.0000 - false_positives_1: 149.0000 - false_negatives_1: 623.0000
Epoch 4/10
236/236 [==============================] - 58s 247ms/step - loss: 0.5044 - accuracy: 0.8806 - true_positives_1: 1651.0000 - true_negatives_1: 7281.0000 - false_positives_1: 89.0000 - false_negatives_1: 399.0000
Epoch 5/10
236/236 [==============================] - 58s 248ms/step - loss: 0.3759 - accuracy: 0.9209 - true

# Calculate metrics (precision, recall, f1-score)

In [ ]:
# get results
python_test_results = model.predict(python_test_dataset.batch(batch_size))

51/51 [==============================] - 4s 40ms/step


In [ ]:
# calculate and output metrics
def getMetrics(y_true, y_pred):
    TP = [0]*5
    TN = [0]*5
    FP = [0]*5
    FN = [0]*5

    results = []

    for i in range(5):
        pred_col = y_pred[:, i].tolist()
        true_col = np.array(y_true.labels.tolist())[:, i].tolist()

        for j in range(len(pred_col)):
            pred_col[j] = int(pred_col[j] > .5)

        for pair in zip(true_col, pred_col):
            if pair[0] == pair[1]:
                if pair[0] == 1:
                    TP[i] += 1
                else:
                    TN[i] += 1
            else:
                if pair[1] == 1:
                    FP[i] += 1
                else:
                    FN[i] += 1
        precision = TP[i]/(TP[i]+FP[i])
        recall = TP[i]/(TP[i]+FN[i])
        f1 = (2*precision*recall)/(precision+recall)
        results.append([precision, recall, f1])


    return results

results = np.array(getMetrics(python_test, python_test_results))
print(results)
print(sum(results[:, 0])/5)
print(sum(results[:, 1])/5)
print(sum(results[:, 2])/5)

[[0.84210526 0.66115702 0.74074074]
 [0.81395349 0.8203125  0.81712062]
 [0.39285714 0.26829268 0.31884058]
 [0.6        0.46875    0.52631579]
 [0.71428571 0.79268293 0.75144509]]
0.672640321734569
0.6022390269098972
0.6308925638395732
